# Imports

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics

In [ ]:
df_car = pd.read_csv('https://raw.githubusercontent.com/pokengineer/DataScience/main/datasets/car_price.csv')
df_car.head(5)

# Analisis del Dataset

In [ ]:
correlation = df_car.corr()
threshold = 0.7
filter = np.abs(correlation["selling_price"]) > threshold
correlation_features = correlation.columns[filter].tolist()
sns.pairplot(df_car[correlation_features], diag_kind = "kde",  hue="selling_price")
plt.show()

# Preprocesamiento de datos

In [ ]:
def preparacion_de_datos(df_input, target, escalar_valores=False):
  df = df_input 
  if escalar_valores: 
    scaler_X = StandardScaler(with_mean=True, with_std=True)
    scaler_X.fit(df.drop(target,axis=1))
    x = pd.DataFrame(scaler_X.transform(df.drop(target,axis=1),), columns = df.drop(target,axis=1).columns )
  else:
    x = df.drop(target,axis=1)
  y = df[target]
  return x,y

In [ ]:
X_car, y_car = preparacion_de_datos( df_car, "target" ,escalar_valores=False)

In [ ]:
# Hacemos el Split 70-30 para train-test
X_train, X_test, y_train, y_test = train_test_split(X_car, y_car,
                                                    test_size=0.3, stratify = y_car, random_state=0)

# Entrenamos los modelos

In [ ]:
from sklearn.linear_model import LinearRegression

reglin = LinearRegression( )
reglin.fit(X_train, y_train)
y_pred_lg = reglin.predict(X_test)


# Mejoramos el modelo elegido con GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters =  {}
clf = GridSearchCV( LinearRegression() , parameters, scoring='roc_auc',cv=5)
clf.fit(X_train, y_train)

In [ ]:
# Print best parameters and best score
print("Best Parameters:", clf.best_params_)
print("Best Score:", clf.best_score_)